In [4]:
import pandas as pd 

In [5]:
data = pd.read_csv('data\prix-carburants-fichier-quotidien-test-ods.csv', sep= ';', on_bad_lines= 'skip')

In [6]:
data.columns

Index(['id', 'cp', 'pop', 'adresse', 'ville', 'horaires', 'rupture',
       'fermeture', 'geom', 'prix_maj', 'prix_id', 'prix_valeur', 'prix_nom',
       'com_arm_code', 'Nom Officiel Commune / Arrondissement Municipal',
       'Code Officiel EPCI', 'Nom Officiel EPCI', 'Code Officiel Département',
       'Nom Officiel Département', 'Code Officiel Région',
       'Nom Officiel Région', 'services_service', 'Nom rupture',
       'Début rupture', 'Fin rupture', 'Automate 24-24 (oui/non)'],
      dtype='object')

Fonction pour définir la "bonne" base de données avec les données qui nous intéressent  
Il faut l'améliorer pour qu'elle élimine les doublons

In [7]:
def data_carb(df, carb):
    data_1 = df[['adresse', 'ville', 'geom', 'prix_valeur', 'prix_nom']]
    d = data_1[data_1.prix_nom == carb]
    return d
    


In [8]:
data_2 = data_carb(data, 'SP95').sort_values('prix_valeur')

In [9]:
data_2

,adresse,ville,geom,prix_valeur,prix_nom
2601,Avenue de l'Orme à Martin,Courcouronnes,"48.631,2.417",1.169,SP95
67001,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
2737,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
38557,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
65471,13 Avenue Ampère,MONTIGNY-LE-BRETONNEUX,"48.798,2.048",1.423,SP95
...,...,...,...,...,...
46107,15 Avenue Duquesne,PARIS,"48.853,2.308",2.600,SP95
53592,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95
58108,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95
43085,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95


Fonction pour calculer la distance à partir de données de longitude et de latitude

In [10]:
from math import cos, sin, acos 

def distance1(p1,p2):
    lat1,long1 = p1
    lat2,long2 = p2
    dlong = long2 - long1
    dist_ang = acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(dlong))
    return(6378137*dist_ang)

Fonction pour renvoyer un data frame avec les 10 plus proches stations essences qui propose le type de carburant recherché   
Majeur problème : les doublons 

In [86]:
def dix_plus_proche_card(df, posi, carb):
    data_1 = data_carb(df, carb).sort_values('prix_valeur')
    pos = list(map(float,posi.split(',')))
    def distancetoposi(p2):
        pos_carb = list(map(float, p2.split(',')))
        return distance1(pos, pos_carb)
    new_df = data_1['geom'].apply(distancetoposi).sort_values().head(10)
    return data_1.loc[new_df.index.to_list()]



    


In [87]:
dix_plus_proche_card(data, '45.45,04.50', 'E10')

,adresse,ville,geom,prix_valeur,prix_nom
13983,Lieu-Dit la Maison Rouge,SAINT-JEAN-BONNEFONDS,"45.443,4.462",1.819,E10
2238,27 Boulevard Fauriat,Saint-Étienne,"45.446,4.407",1.839,E10
4240,27 Boulevard Fauriat,Saint-Étienne,"45.446,4.407",1.839,E10
77947,"140 Rue de la Montat, Quartier de Monthieu",SAINT-ÉTIENNE,"45.442371884059,4.4192750511172",1.699,E10
23785,"140 Rue de la Montat, Quartier de Monthieu",SAINT-ÉTIENNE,"45.442371884059,4.4192750511172",1.699,E10
36484,La Varizelle,Saint-Chamond,"45.462,4.493",1.699,E10
50052,La Varizelle,Saint-Chamond,"45.462,4.493",1.699,E10
13952,100 RUE DE LA MONTAT,Saint-Étienne,"45.44232,4.40992",1.652,E10
29860,50 BD NORMANDIE NIEMEN,Saint-Étienne,"45.437398103719,4.4109830690643",1.699,E10
18956,11 AV DE VERDUN,Saint-Étienne,"45.46519,4.37809",1.652,E10
